In [2]:
pip install imageio


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [3]:
import os
import pandas as pd
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import base64
from PIL import Image
import io
import imageio.v3 as iio
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
csv_file_path = 'Dataset/full_emoji.csv'
df = pd.read_csv(csv_file_path)

In [5]:
def get_emoji_name(image_filename):
    image_id = os.path.splitext(image_filename)[0]
    
    if image_id.isdigit():
        image_id = int(image_id)
        emoji_row = df.iloc[image_id - 1]  
        return emoji_row['name']
    else:
        return "Emoji not found"

In [22]:
image_dir = 'Dataset/image/'

In [23]:
X = []
y = []

In [24]:
for image_file in os.listdir(image_dir):
    if image_file.endswith('.png'):
        image_path = os.path.join(image_dir, image_file)
        
        image = load_img(image_path, target_size=(64, 64)) 
        image = img_to_array(image)
        X.append(image)
        
        emoji_name = get_emoji_name(image_file)
        y.append(emoji_name)

c:\Users\SHIVAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [25]:
X = np.array(X, dtype='float32') / 255.0  
y = np.array(y)

In [26]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [27]:
y_one_hot = to_categorical(y_encoded)

In [28]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(label_encoder.classes_), activation='softmax')
])

c:\Users\SHIVAM\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [29]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
model.fit(X, y_one_hot, epochs=30, batch_size=32)

Epoch 1/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2957
Epoch 2/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2850
Epoch 3/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2844
Epoch 4/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2806
Epoch 5/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 7.8944e-04 - loss: 7.2860
Epoch 6/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2850
Epoch 7/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2850
Epoch 8/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0000e+00 - loss: 7.2792
Epoch 9/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0046 - loss: 7.1790
Epoch 10/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0098 - loss: 6.9288
Epoch 11/30
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.0223 - loss: 6.1031
Epoch 12/30
46/46 ━━━━━━━━━━━━━

In [31]:
model.save('emoji_classifier_model.h5')

In [32]:
np.save('label_classes.npy', label_encoder.classes_)

In [35]:

def predict_emoji_name(image_path):

    image = load_img(image_path, target_size=(64, 64))
    image = img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    
    model = tf.keras.models.load_model('emoji_classifier_model.h5')

    predictions = model.predict(image)
    predicted_label = np.argmax(predictions, axis=1)

    label_classes = np.load('label_classes.npy', allow_pickle=True)
    emoji_name = label_classes[predicted_label][0]
    
    return emoji_name


In [40]:
# Example prediction
image_path = "Dataset/image/18.png"  # Replace with your image path
emoji_name = predict_emoji_name(image_path)
print(f"The predicted emoji name is: {emoji_name}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
The predicted emoji name is: kissing face
